In [1]:
import os
import openai

In [2]:
os.environ['OPENAI_API_KEY'] = 'YOUR_API_KEY'
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
def create_dish_prompt(list_of_ingredients):
    
    prompt = f"Create a detailed recipe based on only the following ingredients: {','.join(list_of_ingredients)}"+ f"Additionally, assign a title starting with 'Recipt Title: 'to this recipe."
    return prompt

In [4]:
recipe = create_dish_prompt(['ham', 'turkey', 'egg', 'bread', 'kimchi'])

In [5]:
response = openai.Completion.create(engine='text-davinci-003', prompt=recipe, max_tokens=512, temperature=0.7)
response

<OpenAIObject text_completion id=cmpl-700MjqwmFccMtNaTha4YTi2xUrZVS at 0x234b1b6aca0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nRecipe Title: Kimchi-Stuffed Ham and Turkey Roll-Ups\n\nIngredients: \n- 2 slices of ham\n- 2 slices of turkey\n- 1 egg, beaten\n- 2 slices of bread\n- 1/4 cup of kimchi, chopped\n\nInstructions:\n1. Preheat oven to 350\u00b0F.\n2. Place ham and turkey slices on a flat surface.\n3. In a bowl, mix together egg and kimchi.\n4. Place one slice of bread on top of each slice of ham and turkey.\n5. Spread kimchi-egg mixture on top of each slice of bread.\n6. Roll up each slice of ham and turkey with the bread and kimchi-egg mixture.\n7. Place the roll-ups in a greased baking dish.\n8. Bake in preheated oven for 20-25 minutes, or until the roll-ups are golden brown.\n9. Serve warm. Enjoy!"
    }
  ],
  "created": 1680235761,
  "id": "cmpl-700MjqwmFccMtNaTha4YTi2xUrZVS",
  "model": "text-da

In [6]:
 print(response['choices'][0]['text'])



Recipe Title: Kimchi-Stuffed Ham and Turkey Roll-Ups

Ingredients: 
- 2 slices of ham
- 2 slices of turkey
- 1 egg, beaten
- 2 slices of bread
- 1/4 cup of kimchi, chopped

Instructions:
1. Preheat oven to 350°F.
2. Place ham and turkey slices on a flat surface.
3. In a bowl, mix together egg and kimchi.
4. Place one slice of bread on top of each slice of ham and turkey.
5. Spread kimchi-egg mixture on top of each slice of bread.
6. Roll up each slice of ham and turkey with the bread and kimchi-egg mixture.
7. Place the roll-ups in a greased baking dish.
8. Bake in preheated oven for 20-25 minutes, or until the roll-ups are golden brown.
9. Serve warm. Enjoy!


In [7]:
result_text = response['choices'][0]['text']

In [8]:
import re #  specifies a set of strings that matches it;

result_text.split('Recipe Title: ')

['\n\n',
 'Kimchi-Stuffed Ham and Turkey Roll-Ups\n\nIngredients: \n- 2 slices of ham\n- 2 slices of turkey\n- 1 egg, beaten\n- 2 slices of bread\n- 1/4 cup of kimchi, chopped\n\nInstructions:\n1. Preheat oven to 350°F.\n2. Place ham and turkey slices on a flat surface.\n3. In a bowl, mix together egg and kimchi.\n4. Place one slice of bread on top of each slice of ham and turkey.\n5. Spread kimchi-egg mixture on top of each slice of bread.\n6. Roll up each slice of ham and turkey with the bread and kimchi-egg mixture.\n7. Place the roll-ups in a greased baking dish.\n8. Bake in preheated oven for 20-25 minutes, or until the roll-ups are golden brown.\n9. Serve warm. Enjoy!']

In [9]:
def extract_title(recipe):
    return re.findall('^.*Recipe Title: .*$', recipe, re.MULTILINE)[0].strip().split('Recipe Title: ')[-1] # last string in part of recipe title

In [10]:
recipe_title = extract_title(result_text) # we are sending this to dalle-2
recipe_title

'Kimchi-Stuffed Ham and Turkey Roll-Ups'

In [11]:
def dalle2_prompt(recipe_title):
    prompt = f"{recipe_title}, professional food photography, 15mm, studio lighting"
    return prompt    

In [12]:
better_image_prompt = dalle2_prompt(recipe_title) # better prompt for image!!!

In [13]:
image_response = openai.Image.create(
    prompt=extract_title(result_text),
    n=1,
    size='1024x1024'
)



better_image_response = openai.Image.create(
    prompt=extract_title(result_text),
    n=1,
    size='1024x1024'
)


In [14]:
image_url = image_response['data'][0]['url']

better_image_url = better_image_response['data'][0]['url']



In [15]:
import requests # need to import request to gain access to the internet
import shutil # utility

# save image to extract, and make image better

def save_image(image_url, file_name):
    # URL --> pic.png
    image_res = requests.get(image_url, stream=True)
    if image_res.status_code == 200:
        with open(file_name, 'wb') as f:
            shutil.copyfileobj(image_res.raw, f)
    else:
        print('ERROR LOADING IMAGE')
    
    return image_res.status_code



def save_better_image(better_image_prompt, file_name):
    # URL --> pic.png
    image_res = requests.get(better_image_url, stream=True)
    if image_res.status_code == 200:
        with open(file_name, 'wb') as f:
            shutil.copyfileobj(image_res.raw, f)
    else:
        print('ERROR LOADING IMAGE')
    
    return image_res.status_code

In [16]:
save_image(image_url, 'example_download_Automatic_recipt_creator.png')

200

In [17]:
save_better_image(better_image_prompt, 'example_download_Automatic_recipt_creator_better.png')

200